# Plant Disease Recognition - Model Training (FIXED)

### Step 1: Setup and Download Data

In [ ]:
!wget -O "dataset.zip" "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/file_downloaded"

!unzip -q /content/dataset.zip

!pip install -q split-folders

### Step 2: Import Libraries and Define Constants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import splitfolders

# --- Configuration ---
RAW_DATA_DIR = "/content/Plant_leave_diseases_dataset_with_augmentation"
OUTPUT_DATA_DIR = "/content/dataset"
TRAIN_DIR = "/content/dataset/train"
VALIDATION_DIR = "/content/dataset/val"
TEST_DIR = "/content/dataset/test"

BATCH_SIZE = 32
# CRITICAL FIX: Ensure image size is (160, 160)
IMG_SIZE = (160, 160)
# CRITICAL FIX: Shape must be (160, 160, 3) for RGB
IMG_SHAPE = IMG_SIZE + (3,)

INITIAL_EPOCHS = 6
FINE_TUNE_EPOCHS = 10
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
FINE_TUNE_AT = 100

### Step 3: Split Data

In [ ]:
splitfolders.ratio(RAW_DATA_DIR, output=OUTPUT_DATA_DIR, seed=1337, ratio=(.8, .1,.1))

### Step 4: Data Preprocessing

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(TRAIN_DIR,
                                                                shuffle=True,
                                                                batch_size=BATCH_SIZE,
                                                                image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(VALIDATION_DIR,
                                                                      shuffle=True,
                                                                      batch_size=BATCH_SIZE,
                                                                      image_size=IMG_SIZE)

test_dataset = tf.keras.utils.image_dataset_from_directory(TEST_DIR,
                                                               batch_size=BATCH_SIZE,
                                                               image_size=IMG_SIZE)

class_names = train_dataset.class_names
print(f"Found {len(class_names)} classes.")

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Step 5: Build the Model

In [ ]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model
base_model = tf.keras.applications.EfficientNetB4(
    input_shape=IMG_SHAPE,  # This now correctly uses (160, 160, 3)
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False # Freeze the base

# Build the full model
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

# Compile for feature extraction
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')])

model.summary()

### Step 6: Train (Feature Extraction)

In [ ]:
history = model.fit(train_dataset,
                        epochs=INITIAL_EPOCHS,
                        validation_data=validation_dataset)

### Step 7: Fine-Tuning

In [ ]:
base_model.trainable = True

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:FINE_TUNE_AT]:
  layer.trainable = False

# Re-compile the model for fine-tuning with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')])

model.summary()

# Continue training
history_fine = model.fit(train_dataset,
                         epochs=TOTAL_EPOCHS,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

### Step 8: Evaluate and Save

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

# Save the new, correct model
NEW_MODEL_NAME = "plant_disease_model_V3.keras"
model.save(NEW_MODEL_NAME)
print(f"Model saved as {NEW_MODEL_NAME}")

# Optional: Save to Google Drive
try:
  from google.colab import drive
  drive.mount('/content/drive')
  model.save(f"/content/drive/MyDrive/{NEW_MODEL_NAME}")
  print(f"Model also saved to Google Drive: /content/drive/MyDrive/{NEW_MODEL_NAME}")
except Exception as e:
  print(f"Could not save to Google Drive: {e}")